# GnomAD_df class use example
After each cell There is an explanation of what it is doing

In [1]:
import sys 
import os
sys.path.append(os.path.abspath("code"))
from GnomAD_df_class import GnomAD_df


### explanation:
The first two imports are only for loading the class from it's location.<br> they are not necessery

In [2]:
gdf = GnomAD_df(path='data/main.prq',
                peak_file='data/hATAC_mATAC_peak.bed.gz',
                remove_unkown=True,
                remove_phased_gt=True,
                only_peak_variants=True)
gdf.get_table().head()

applying filters
removing variants outside of peak interval
removed unkwon genotypes (e.g "./.")
replaced phased genotype (e.g "0|1" -> "0/1")
getting table


INTERVAL_ID AS22WG009:GT  AS22WG009:DP  \
CHROM POS    ALT REF                                                   
chr1  634112 C   T         PGRA.SER.GRA_1          0/1          86.0   
      778639 G   A    PGRA.SER.GRA.PSER_2          NaN           NaN   
      779047 A   G    PGRA.SER.GRA.PSER_2          1/1          34.0   
      826893 A   G    PGRA.SER.GRA.PSER_3          1/1          30.0   
      827209 C   G    PGRA.SER.GRA.PSER_3          1/1          25.0   

                     AS22WG010:GT  AS22WG010:DP AS22WG011:GT  AS22WG011:DP  \
CHROM POS    ALT REF                                                         
chr1  634112 C   T            0/1          46.0          0/1          46.0   
      778639 G   A            NaN           NaN          NaN           NaN   
      779047 A   G            1/1          20.0          1/1          21.0   
      826893 A   G            0/1          25.0          1/1          22.0   
      827209 C   G            0/1          35.0          1/1          18.0   

                     AS22WG023:GT  AS22WG023:DP AS22WG024:GT  ...  \
CHROM POS    ALT REF                                          ...   
chr1  634112 C   T            0/1          49.0          0/1  ...   
      778639 G   A            NaN           NaN          NaN  ...   
      779047 A   G            1/1          27.0          1/1  ...   
      826893 A   G            1/1          17.0          1/1  ...   
      827209 C   G            1/1          26.0          1/1  ...   

                      PKGen166:GT PKGen166:DP  PKGen177:GT PKGen177:DP  \
CHROM POS    ALT REF                                                     
chr1  634112 C   T            0/1       204.0          1/1       735.0   
      778639 G   A            NaN         NaN          NaN         NaN   
      779047 A   G            1/1        26.0          1/1        27.0   
      826893 A   G            1/1        30.0          1/1        36.0   
      827209 C   G            1/1        43.0          1/1        43.0   

                      PKGen186:GT PKGen186:DP  PKGen172:GT PKGen172:DP  \
CHROM POS    ALT REF                                                     
chr1  634112 C   T            0/1       138.0          1/1       290.0   
      778639 G   A            NaN         NaN          NaN         NaN   
      779047 A   G            1/1        24.0          1/1        23.0   
      826893 A   G            1/1        36.0          1/1        26.0   
      827209 C   G            0/1        31.0          0/1        29.0   

                      PKGen187:GT PKGen187:DP  
CHROM POS    ALT REF                           
chr1  634112 C   T            NaN         NaN  
      778639 G   A            NaN         NaN  
      779047 A   G            1/1        46.0  
      826893 A   G            1/1        36.0  
      827209 C   G            1/1        28.0  

[5 rows x 76 columns]

### Explanation:
The class constructor have only one mandatory variable, the path to the parquet file. <br>
the rest of the argument, except the peak_file (it can be left empty), are the deafult argument. <br>
Variable description:<br>
- peak_file - bed file which contains the following columns - CHROM,FROM,TO,INTERVAL_ID,  (w/o header)
- remove_unknown - add a filter which removes unkwon genotypes (e.g "./.")' 
- remove_phased_gt -  add a filter which replace phased genotype (e.g "0|1" -> "0/1")
- only_peak_variants -  add a filter which removes variants outside of peak interval 
<br>

Whenever get_table or any other function that requeires to apply the filters, only then the class creates a sub dataframe which the filters apllied on it. <br>
During the filteration process, the filter's descriptions will be printed, unless verbos=False. <br>

In [3]:
gdf.reset_table(remove_phased_gt=False,
    remove_unkown=False,
    only_peak_variants=False)
gdf.get_table().head()


applying filters
getting table


INTERVAL_ID AS22WG009:GT  AS22WG009:DP AS22WG010:GT  \
CHROM POS   ALT REF                                                       
chr1  10117 A   C           NaN          NaN           NaN          NaN   
      10141 A   C           NaN          NaN           NaN          NaN   
      10146 A   AC          NaN          NaN           NaN          NaN   
      10151 A   T           NaN          NaN           NaN          NaN   
      10153 T   A           NaN          NaN           NaN          NaN   

                     AS22WG010:DP AS22WG011:GT  AS22WG011:DP AS22WG023:GT  \
CHROM POS   ALT REF                                                         
chr1  10117 A   C             NaN          NaN           NaN          NaN   
      10141 A   C             NaN          NaN           NaN          NaN   
      10146 A   AC            NaN          NaN           NaN          NaN   
      10151 A   T             NaN          NaN           NaN          NaN   
      10153 T   A             NaN          NaN           NaN          NaN   

                     AS22WG023:DP AS22WG024:GT  ...  PKGen166:GT PKGen166:DP  \
CHROM POS   ALT REF                             ...                            
chr1  10117 A   C             NaN          NaN  ...          NaN         NaN   
      10141 A   C             NaN          NaN  ...          NaN         NaN   
      10146 A   AC            NaN          1/1  ...          NaN         NaN   
      10151 A   T             NaN          NaN  ...          NaN         NaN   
      10153 T   A             NaN          NaN  ...          NaN         NaN   

                     PKGen177:GT PKGen177:DP  PKGen186:GT PKGen186:DP  \
CHROM POS   ALT REF                                                     
chr1  10117 A   C            NaN         NaN          NaN         NaN   
      10141 A   C            NaN         NaN          NaN         NaN   
      10146 A   AC           NaN         NaN          NaN         NaN   
      10151 A   T            NaN         NaN          NaN         NaN   
      10153 T   A            NaN         NaN          NaN         NaN   

                     PKGen172:GT PKGen172:DP  PKGen187:GT PKGen187:DP  
CHROM POS   ALT REF                                                    
chr1  10117 A   C            NaN         NaN          NaN         NaN  
      10141 A   C            NaN         NaN          NaN         NaN  
      10146 A   AC           NaN         NaN          NaN         NaN  
      10151 A   T            NaN         NaN          NaN         NaN  
      10153 T   A            NaN         NaN          NaN         NaN  

[5 rows x 76 columns]

### Explanation:
In the cell below, No filter was applied. The original table is shown. <br>
Now lets filter it up!

In [4]:
gdf.remove_non_peak_variants()
gdf.remove_phasing()
gdf.remove_unkown()
gdf.get_table().head()

applying filters
removing variants outside of peak interval
replaced phased genotype (e.g "0|1" -> "0/1")
removed unkwon genotypes (e.g "./.")
getting table


INTERVAL_ID AS22WG009:GT  AS22WG009:DP  \
CHROM POS    ALT REF                                                   
chr1  634112 C   T         PGRA.SER.GRA_1          0/1          86.0   
      778639 G   A    PGRA.SER.GRA.PSER_2          NaN           NaN   
      779047 A   G    PGRA.SER.GRA.PSER_2          1/1          34.0   
      826893 A   G    PGRA.SER.GRA.PSER_3          1/1          30.0   
      827209 C   G    PGRA.SER.GRA.PSER_3          1/1          25.0   

                     AS22WG010:GT  AS22WG010:DP AS22WG011:GT  AS22WG011:DP  \
CHROM POS    ALT REF                                                         
chr1  634112 C   T            0/1          46.0          0/1          46.0   
      778639 G   A            NaN           NaN          NaN           NaN   
      779047 A   G            1/1          20.0          1/1          21.0   
      826893 A   G            0/1          25.0          1/1          22.0   
      827209 C   G            0/1          35.0          1/1          18.0   

                     AS22WG023:GT  AS22WG023:DP AS22WG024:GT  ...  \
CHROM POS    ALT REF                                          ...   
chr1  634112 C   T            0/1          49.0          0/1  ...   
      778639 G   A            NaN           NaN          NaN  ...   
      779047 A   G            1/1          27.0          1/1  ...   
      826893 A   G            1/1          17.0          1/1  ...   
      827209 C   G            1/1          26.0          1/1  ...   

                      PKGen166:GT PKGen166:DP  PKGen177:GT PKGen177:DP  \
CHROM POS    ALT REF                                                     
chr1  634112 C   T            0/1       204.0          1/1       735.0   
      778639 G   A            NaN         NaN          NaN         NaN   
      779047 A   G            1/1        26.0          1/1        27.0   
      826893 A   G            1/1        30.0          1/1        36.0   
      827209 C   G            1/1        43.0          1/1        43.0   

                      PKGen186:GT PKGen186:DP  PKGen172:GT PKGen172:DP  \
CHROM POS    ALT REF                                                     
chr1  634112 C   T            0/1       138.0          1/1       290.0   
      778639 G   A            NaN         NaN          NaN         NaN   
      779047 A   G            1/1        24.0          1/1        23.0   
      826893 A   G            1/1        36.0          1/1        26.0   
      827209 C   G            0/1        31.0          0/1        29.0   

                      PKGen187:GT PKGen187:DP  
CHROM POS    ALT REF                           
chr1  634112 C   T            NaN         NaN  
      778639 G   A            NaN         NaN  
      779047 A   G            1/1        46.0  
      826893 A   G            1/1        36.0  
      827209 C   G            1/1        28.0  

[5 rows x 76 columns]

### Explanation:
Magic! We got the same table from 2 cells ago! <br>
Notice that the order of filteration changed. This is important, if we would like to remove a filter, we can do it by it's index. <br>
Of course there is a simple way to know the filters index... <br>

Lets see:

In [5]:
gdf.reset_table(remove_phased_gt=True,
    remove_unkown=True,
    only_peak_variants=True)
display(gdf.get_table().head())
gdf.filter_dp(31)
print('\n')
print("look on AS22WG009:GT, 4th & 5th row, \n it's written NaN since we filtered according to dp thrshold of 31 and above ")
print('\n')
display(gdf.get_table().head())


applying filters
removing variants outside of peak interval
replaced phased genotype (e.g "0|1" -> "0/1")
removed unkwon genotypes (e.g "./.")
getting table


INTERVAL_ID AS22WG009:GT  AS22WG009:DP  \
CHROM POS    ALT REF                                                   
chr1  634112 C   T         PGRA.SER.GRA_1          0/1          86.0   
      778639 G   A    PGRA.SER.GRA.PSER_2          NaN           NaN   
      779047 A   G    PGRA.SER.GRA.PSER_2          1/1          34.0   
      826893 A   G    PGRA.SER.GRA.PSER_3          1/1          30.0   
      827209 C   G    PGRA.SER.GRA.PSER_3          1/1          25.0   

                     AS22WG010:GT  AS22WG010:DP AS22WG011:GT  AS22WG011:DP  \
CHROM POS    ALT REF                                                         
chr1  634112 C   T            0/1          46.0          0/1          46.0   
      778639 G   A            NaN           NaN          NaN           NaN   
      779047 A   G            1/1          20.0          1/1          21.0   
      826893 A   G            0/1          25.0          1/1          22.0   
      827209 C   G            0/1          35.0          1/1          18.0   

                     AS22WG023:GT  AS22WG023:DP AS22WG024:GT  ...  \
CHROM POS    ALT REF                                          ...   
chr1  634112 C   T            0/1          49.0          0/1  ...   
      778639 G   A            NaN           NaN          NaN  ...   
      779047 A   G            1/1          27.0          1/1  ...   
      826893 A   G            1/1          17.0          1/1  ...   
      827209 C   G            1/1          26.0          1/1  ...   

                      PKGen166:GT PKGen166:DP  PKGen177:GT PKGen177:DP  \
CHROM POS    ALT REF                                                     
chr1  634112 C   T            0/1       204.0          1/1       735.0   
      778639 G   A            NaN         NaN          NaN         NaN   
      779047 A   G            1/1        26.0          1/1        27.0   
      826893 A   G            1/1        30.0          1/1        36.0   
      827209 C   G            1/1        43.0          1/1        43.0   

                      PKGen186:GT PKGen186:DP  PKGen172:GT PKGen172:DP  \
CHROM POS    ALT REF                                                     
chr1  634112 C   T            0/1       138.0          1/1       290.0   
      778639 G   A            NaN         NaN          NaN         NaN   
      779047 A   G            1/1        24.0          1/1        23.0   
      826893 A   G            1/1        36.0          1/1        26.0   
      827209 C   G            0/1        31.0          0/1        29.0   

                      PKGen187:GT PKGen187:DP  
CHROM POS    ALT REF                           
chr1  634112 C   T            NaN         NaN  
      778639 G   A            NaN         NaN  
      779047 A   G            1/1        46.0  
      826893 A   G            1/1        36.0  
      827209 C   G            1/1        28.0  

[5 rows x 76 columns]



look on AS22WG009:GT, 4th & 5th row, 
 it's written NaN since we filtered according to dp thrshold of 31 and above 


applying filters
removing variants outside of peak interval
replaced phased genotype (e.g "0|1" -> "0/1")
removed unkwon genotypes (e.g "./.")
Removing variants with reading depth 31 or below
getting table


INTERVAL_ID AS22WG009:GT  AS22WG009:DP  \
CHROM POS    ALT REF                                                   
chr1  634112 C   T         PGRA.SER.GRA_1          0/1          86.0   
      778639 G   A    PGRA.SER.GRA.PSER_2          NaN           NaN   
      779047 A   G    PGRA.SER.GRA.PSER_2          1/1          34.0   
      826893 A   G    PGRA.SER.GRA.PSER_3          NaN          30.0   
      827209 C   G    PGRA.SER.GRA.PSER_3          NaN          25.0   

                     AS22WG010:GT  AS22WG010:DP AS22WG011:GT  AS22WG011:DP  \
CHROM POS    ALT REF                                                         
chr1  634112 C   T            0/1          46.0          0/1          46.0   
      778639 G   A            NaN           NaN          NaN           NaN   
      779047 A   G            NaN          20.0          NaN          21.0   
      826893 A   G            NaN          25.0          NaN          22.0   
      827209 C   G            0/1          35.0          NaN          18.0   

                     AS22WG023:GT  AS22WG023:DP AS22WG024:GT  ...  \
CHROM POS    ALT REF                                          ...   
chr1  634112 C   T            0/1          49.0          0/1  ...   
      778639 G   A            NaN           NaN          NaN  ...   
      779047 A   G            NaN          27.0          NaN  ...   
      826893 A   G            NaN          17.0          NaN  ...   
      827209 C   G            NaN          26.0          1/1  ...   

                      PKGen166:GT PKGen166:DP  PKGen177:GT PKGen177:DP  \
CHROM POS    ALT REF                                                     
chr1  634112 C   T            0/1       204.0          1/1       735.0   
      778639 G   A            NaN         NaN          NaN         NaN   
      779047 A   G            NaN        26.0          NaN        27.0   
      826893 A   G            NaN        30.0          1/1        36.0   
      827209 C   G            1/1        43.0          1/1        43.0   

                      PKGen186:GT PKGen186:DP  PKGen172:GT PKGen172:DP  \
CHROM POS    ALT REF                                                     
chr1  634112 C   T            0/1       138.0          1/1       290.0   
      778639 G   A            NaN         NaN          NaN         NaN   
      779047 A   G            NaN        24.0          NaN        23.0   
      826893 A   G            1/1        36.0          NaN        26.0   
      827209 C   G            NaN        31.0          NaN        29.0   

                      PKGen187:GT PKGen187:DP  
CHROM POS    ALT REF                           
chr1  634112 C   T            NaN         NaN  
      778639 G   A            NaN         NaN  
      779047 A   G            1/1        46.0  
      826893 A   G            1/1        36.0  
      827209 C   G            NaN        28.0  

[5 rows x 76 columns]

Since our real threshold is 10, we need to fix it. <br>
1st we need to find the index of the filter

In [6]:
print("current filters:")
gdf.print_filters()
print("\n lets remove filter no 3")
gdf.remove_filter(3)
print('\n current filters: ')
gdf.get_table().head()

current filters:
0. removing variants outside of peak interval
1. replaced phased genotype (e.g "0|1" -> "0/1")
2. removed unkwon genotypes (e.g "./.")
3. Removing variants with reading depth 31 or below

 lets remove filter no 3

 current filters: 
applying filters
removing variants outside of peak interval
replaced phased genotype (e.g "0|1" -> "0/1")
removed unkwon genotypes (e.g "./.")
getting table


INTERVAL_ID AS22WG009:GT  AS22WG009:DP  \
CHROM POS    ALT REF                                                   
chr1  634112 C   T         PGRA.SER.GRA_1          0/1          86.0   
      778639 G   A    PGRA.SER.GRA.PSER_2          NaN           NaN   
      779047 A   G    PGRA.SER.GRA.PSER_2          1/1          34.0   
      826893 A   G    PGRA.SER.GRA.PSER_3          1/1          30.0   
      827209 C   G    PGRA.SER.GRA.PSER_3          1/1          25.0   

                     AS22WG010:GT  AS22WG010:DP AS22WG011:GT  AS22WG011:DP  \
CHROM POS    ALT REF                                                         
chr1  634112 C   T            0/1          46.0          0/1          46.0   
      778639 G   A            NaN           NaN          NaN           NaN   
      779047 A   G            1/1          20.0          1/1          21.0   
      826893 A   G            0/1          25.0          1/1          22.0   
      827209 C   G            0/1          35.0          1/1          18.0   

                     AS22WG023:GT  AS22WG023:DP AS22WG024:GT  ...  \
CHROM POS    ALT REF                                          ...   
chr1  634112 C   T            0/1          49.0          0/1  ...   
      778639 G   A            NaN           NaN          NaN  ...   
      779047 A   G            1/1          27.0          1/1  ...   
      826893 A   G            1/1          17.0          1/1  ...   
      827209 C   G            1/1          26.0          1/1  ...   

                      PKGen166:GT PKGen166:DP  PKGen177:GT PKGen177:DP  \
CHROM POS    ALT REF                                                     
chr1  634112 C   T            0/1       204.0          1/1       735.0   
      778639 G   A            NaN         NaN          NaN         NaN   
      779047 A   G            1/1        26.0          1/1        27.0   
      826893 A   G            1/1        30.0          1/1        36.0   
      827209 C   G            1/1        43.0          1/1        43.0   

                      PKGen186:GT PKGen186:DP  PKGen172:GT PKGen172:DP  \
CHROM POS    ALT REF                                                     
chr1  634112 C   T            0/1       138.0          1/1       290.0   
      778639 G   A            NaN         NaN          NaN         NaN   
      779047 A   G            1/1        24.0          1/1        23.0   
      826893 A   G            1/1        36.0          1/1        26.0   
      827209 C   G            0/1        31.0          0/1        29.0   

                      PKGen187:GT PKGen187:DP  
CHROM POS    ALT REF                           
chr1  634112 C   T            NaN         NaN  
      778639 G   A            NaN         NaN  
      779047 A   G            1/1        46.0  
      826893 A   G            1/1        36.0  
      827209 C   G            1/1        28.0  

[5 rows x 76 columns]

Yay :) the table looks similar to the original! <br>
<br>
## Example : Filtering the table according to thresholds

After the short torturial above, see how easily is to achive a table which contains variants acoording to given thresholds.<br>
Let's say we want only variants with reading depth (DP) > 10 & gnomAD allele frequency < 0.01 <br>
First, lts see how many variant each sample have before applying the filters:    

In [ ]:
gdf.reset_table() # make sure we cleaned all the mess from the cell above

print('first step: getting a boolean table, which suggest if a sample have a variant or not')
bool_df = gdf.bool_variant_df()
display(bool_df.head())
print('Now lets count how many variants each sample have -')
boo_df.drop(columns=['AF','INTERVAL_ID']).sum()

first step: getting a boolean table, which suggest if a sample have a variant or not


In [11]:
bool_df

AS22WG009  AS22WG010  AS22WG011  AS22WG023  \
CHROM POS       ALT REF                                               
chr1  634112    C   T         True       True       True       True   
      778639    G   A        False      False      False      False   
      779047    A   G         True       True       True       True   
      826893    A   G         True       True       True       True   
      827209    C   G         True       True       True       True   
...                            ...        ...        ...        ...   
chrX  152897990 T   C        False      False      False      False   
      153381374 G   C        False      False      False      False   
      153727301 G   A        False      False      False      False   
      153901846 A   G        False      False      False      False   
      153972434 A   G        False      False      False      False   

                         AS22WG024  AS22WG004  AS22WG005  AS22WG006  \
CHROM POS       ALT REF                                               
chr1  634112    C   T         True       True       True       True   
      778639    G   A        False      False      False      False   
      779047    A   G         True       True       True       True   
      826893    A   G         True       True       True       True   
      827209    C   G         True       True       True       True   
...                            ...        ...        ...        ...   
chrX  152897990 T   C        False      False      False      False   
      153381374 G   C        False      False      False      False   
      153727301 G   A        False      False      False      False   
      153901846 A   G        False      False      False      False   
      153972434 A   G        False      False      False      False   

                         AS22WG007  AS22WG008  ...  PKGen171  PKGen185  \
CHROM POS       ALT REF                        ...                       
chr1  634112    C   T         True       True  ...      True      True   
      778639    G   A        False      False  ...     False      True   
      779047    A   G         True       True  ...      True      True   
      826893    A   G         True       True  ...      True     False   
      827209    C   G         True       True  ...      True     False   
...                            ...        ...  ...       ...       ...   
chrX  152897990 T   C        False      False  ...     False     False   
      153381374 G   C        False      False  ...     False     False   
      153727301 G   A        False      False  ...     False     False   
      153901846 A   G        False      False  ...     False     False   
      153972434 A   G        False      False  ...     False     False   

                         PKGen182  PKGen166  PKGen177  PKGen186  PKGen172  \
CHROM POS       ALT REF                                                     
chr1  634112    C   T        True      True      True      True      True   
      778639    G   A       False     False     False     False     False   
      779047    A   G        True      True      True      True      True   
      826893    A   G        True      True      True      True      True   
      827209    C   G        True      True      True      True      True   
...                           ...       ...       ...       ...       ...   
chrX  152897990 T   C       False     False     False     False     False   
      153381374 G   C       False     False     False     False     False   
      153727301 G   A       False     False     False     False     False   
      153901846 A   G       False     False     False     False     False   
      153972434 A   G       False     False     False     False     False   

                         PKGen187        AF          INTERVAL_ID  
CHROM POS       ALT REF                                           
chr1  634112    C   T       False  0.581946       PGRA.SER.GRA_1  
      77863

In [8]:


DP_t = 10 
AF_t = 0.01
gdf.filter_dp(DP_t)
gdf.filter_af(AF_t, remove_unkwon=True)
gdf.get_table()

AttributeError: 'GnomAD_df' object has no attribute 'filter_DP'